In [193]:
import math
import random
from copy import deepcopy
from collections import namedtuple

In [16]:
Customer = namedtuple("Customer", ['index', 'demand', 'x', 'y'])
Problem = namedtuple("Problem", ['vehicle_count', 'vehicle_capacity', 'customers', 'depot'])

In [17]:
file_location = "../data/vrp_5_4_1"
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()

lines = input_data.split('\n')
parts = lines[0].split()
customer_count = int(parts[0])
vehicle_count = int(parts[1])
vehicle_capacity = int(parts[2])
    
customers = []
for i in range(1, customer_count+1):
    line = lines[i]
    parts = line.split()
    customers.append(Customer(i-1, int(parts[0]), float(parts[1]), float(parts[2])))

#the depot is always the first customer in the input
depot = customers[0] 

problem = Problem(vehicle_count, vehicle_capacity, customers, depot)

In [4]:
def dist(customer1, customer2):
    return math.sqrt((customer1.x - customer2.x)**2 + (customer1.y - customer2.y)**2)

In [5]:
print (customers)
print("---------------")
print(depot)

[Customer(index=0, demand=0, x=0.0, y=0.0), Customer(index=1, demand=3, x=0.0, y=10.0), Customer(index=2, demand=3, x=-10.0, y=10.0), Customer(index=3, demand=3, x=0.0, y=-10.0), Customer(index=4, demand=3, x=10.0, y=-10.0)]
---------------
Customer(index=0, demand=0, x=0.0, y=0.0)


In [6]:
def obj(vehicle_tours, ):
    obj = 0
    for v in range(0, vehicle_count):
        vehicle_tour = vehicle_tours[v]
        if len(vehicle_tour) > 0:
            obj += dist(depot,vehicle_tour[0])
            for i in range(0, len(vehicle_tour)-1):
                obj += dist(vehicle_tour[i],vehicle_tour[i+1])
            obj += dist(vehicle_tour[-1],depot)
    return obj

In [80]:
def generate_solution(problem):
    vehicle_tours = [[] for _ in range(problem.vehicle_count)]
    remaining_customers = set(problem.customers)
    remaining_customers.remove(problem.depot)
    vehicle_loads = [0 for _ in range(problem.vehicle_count)]

    while len(remaining_customers) > 0:
        customer = remaining_customers.pop()
        for i in range(problem.vehicle_count):
            if vehicle_loads[i] + customer.demand <= problem.vehicle_capacity:
                vehicle_tours[i].append(customer)
                vehicle_loads[i] += customer.demand
                break
    return vehicle_tours

In [187]:
s = generate_solution(problem)
obj(s)

80.6449510224598

In [118]:
def is_valid(problem, solution):
    
    customer_count = [0 for _ in range(len(problem.customers))]
    capacity_count = [0 for _ in range(problem.vehicle_count)]
    for car_ix, tour in enumerate(solution):
        for cus in tour:
            customer_count[cus.index] += 1
            capacity_count[car_ix] += cus.demand
            
    ## check is all the customers are visited once and only once:
    for cus_cnt in customer_count[1:]: ## ignore depot
        if cus_cnt != 1: #  "One or more customer is not served or is served more than once."
            return false
    
    ## Check that the capacity is not exceeded
    for capa in capacity_count:
        if capa > problem.vehicle_capacity: ## capacity exceeded!
            return False
    return True

In [119]:
is_valid(problem, s)

True

## Neighborhood

Here, we define the neighborhood of a solution as every new solution (valid or not) reachable by moving a customer from one tour to another one or in the same tour but at another position

In [194]:
def swap_cus_same_tour(solution, tour_ix, pos1, pos2):
    """Swap 2 position within the same tour. Return the new solution after the move or the solution 
    if an index is illegally accessed.
    """
    solution_cpy = deepcopy(solution)
    try: 
        temp = solution_cpy[tour_ix][pos1]
        solution_cpy[tour_ix][pos1] = solution_cpy[tour_ix][pos2]
        solution_cpy[tour_ix][pos2] = temp
    except:
        return solution
    
    return solution_cpy

In [195]:
def swap_cus_different_tour(solution, tour1_ix, tour2_ix, pos1, pos2):
    """Move a customer from one tour to another. Return the new solution after the move or the solution 
    if an index is illegally accessed.
    """
    solution_cpy = deepcopy(solution)
    try:
        cus = solution_cpy[tour1_ix].pop(pos1)
        solution_cpy[tour2_ix].insert(pos2, cus)
    except:
        return solution
    return solution_cpy

In [196]:
s

[[Customer(index=1, demand=3, x=0.0, y=10.0),
  Customer(index=2, demand=3, x=-10.0, y=10.0)],
 [Customer(index=3, demand=3, x=0.0, y=-10.0)],
 [],
 [Customer(index=4, demand=3, x=10.0, y=-10.0)]]

In [197]:
k = swap_cus_different_tour(s, 0, 3, 0, 0)

In [198]:
obj(k)

94.78708664619074

In [199]:
s

[[Customer(index=1, demand=3, x=0.0, y=10.0),
  Customer(index=2, demand=3, x=-10.0, y=10.0)],
 [Customer(index=3, demand=3, x=0.0, y=-10.0)],
 [],
 [Customer(index=4, demand=3, x=10.0, y=-10.0)]]

In [200]:
k

[[Customer(index=2, demand=3, x=-10.0, y=10.0)],
 [Customer(index=3, demand=3, x=0.0, y=-10.0)],
 [],
 [Customer(index=1, demand=3, x=0.0, y=10.0),
  Customer(index=4, demand=3, x=10.0, y=-10.0)]]